In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functools import reduce
import os
import sys
sys.path.insert(0, '../../src/')
import data_loader
pd.set_option('display.max_columns', None)

In [23]:
# seasons_data = data_loader.load_data()
# season_2018 = data_loader.load_data(seasons=["2018-19"])
# game_data = data_loader.load_game_data(seasons=["2018-19"])
# seasons_game_data = data_loader.load_game_data()

Loaded PBP-data


In [21]:
seasons_data = data_loader.load_data()
seasons_data

,Unnamed: 0,EVENTMSGACTIONTYPE,EVENTMSGTYPE,EVENTNUM,GAME_ID,HOMEDESCRIPTION,PCTIMESTRING,PERIOD,PERSON1TYPE,PERSON2TYPE,PERSON3TYPE,PLAYER1_ID,PLAYER1_NAME,PLAYER1_TEAM_ABBREVIATION,PLAYER1_TEAM_CITY,PLAYER1_TEAM_ID,PLAYER1_TEAM_NICKNAME,PLAYER2_ID,PLAYER2_NAME,PLAYER2_TEAM_ABBREVIATION,PLAYER2_TEAM_CITY,PLAYER2_TEAM_ID,PLAYER2_TEAM_NICKNAME,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ABBREVIATION,PLAYER3_TEAM_CITY,PLAYER3_TEAM_ID,PLAYER3_TEAM_NICKNAME,SCORE,SCOREMARGIN,VISITORDESCRIPTION,WCTIMESTRING,season_name
0,0,0,PERIOD_BEGIN,2,21800794,NaN,12:00,1,0.0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9:11 PM,2018-19
1,1,0,JUMP_BALL,4,21800794,Jump Ball Ayton vs. Faried: Tip to Okobo,12:00,1,HOME_PLAYER,VISITOR_PLAYER,HOME_PLAYER,1629028,Deandre Ayton,PHX,Phoenix,1610612756.0,Suns,202702,Kenneth Faried,HOU,Houston,1610612745.0,Rockets,1629059,Elie Okobo,PHX,Phoenix,1610612756.0,Suns,NaN,NaN,NaN,9:11 PM,2018-19
2,2,1,TURNOVER,7,21800794,Jackson Bad Pass Turnover (P1.T1),11:39,1,HOME_PLAYER,VISITOR_PLAYER,0,1628367,Josh Jackson,PHX,Phoenix,1610612756.0,Suns,201935,James Harden,HOU,Houston,1610612745.0,Rockets,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Harden STEAL (1 STL),9:11 PM,2018-19
3,3,Driving Layup,FIELD_GOAL_MADE,9,21800794,NaN,11:31,1,VISITOR_PLAYER,0,0,201935,James Harden,HOU,Houston,1610612745.0,Rockets,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,2 - 0,-2,Harden 3' Driving Layup (2 PTS),9:12 PM,2018-19
4,4,2,FOUL,10,21800794,Bridges S.FOUL (P1.T1) (E.Dalen),11:31,1,HOME_PLAYER,VISITOR_PLAYER,TIMEOUT,1628969,Mikal Bridges,PHX,Phoenix,1610612756.0,Suns,201935,James Harden,HOU,Houston,1610612745.0,Rockets,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9:12 PM,2018-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582463,431,1,TIMEOUT,634,21801037,NaN,0:04,4,VISITOR_TEAM,0,0,1610612739,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cavaliers Timeout: Regular (Reg.6 Short 0),10:49 PM,2018-19
582464,432,0,SUBSTITUTION,635,21801037,NaN,0:04,4,VISITOR_PLAYER,VISITOR_PLAYER,0,1627790,Ante Zizic,CLE,Cleveland,1610612739.0,Cavaliers,203903,Jordan Clarkson,CLE,Cleveland,1610612739.0,Cavaliers,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SUB: Clarkson FOR Zizic,10:50 PM,2018-19
582465,433,Fadeaway Jumper,FIELD_GOAL_MISSED,638,21801037,NaN,0:02,4,VISITOR_PLAYER,0,0,203917,Nik Stauskas,CLE,Cleveland,1610612739.0,Cavaliers,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MISS Stauskas 24' 3PT Fadeaway Jumper,10:51 PM,2018-19
582466,434,live,REBOUND,639,21801037,Kleber REBOUND (Off:5 Def:7),0:02,4,HOME_PLAYER,0,0,1628467,Maxi Kleber,DAL,Dallas,1610612742.0,Mavericks,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10:51 PM,2018-19


In [8]:
GAME_ID_STR = "GAME_ID"
pbp_grouped = seasons_data.groupby(GAME_ID_STR)
game_data = pbp_grouped.size().to_frame(name="play_count")

game_data["HOME TEAM ID"] = pbp_grouped.apply(lambda x: x[x["PERSON1TYPE"] == 'HOME_PLAYER']["PLAYER1_TEAM_ID"].iloc[0])
game_data["VISITOR TEAM ID"] = pbp_grouped.apply(lambda x: x[x["PERSON1TYPE"] == 'VISITOR_PLAYER']["PLAYER1_TEAM_ID"].iloc[0])
game_data[["HOME_PLAYER_1", "HOME_PLAYER_2", "HOME_PLAYER_3", "HOME_PLAYER_4", "HOME_PLAYER_5", "VISITOR_PLAYER_1", "VISITOR_PLAYER_2", "VISITOR_PLAYER_3", "VISITOR_PLAYER_4", "VISITOR_PLAYER_5"]] = np.nan

errors = []

In [1]:
for year in range(0, 19):

    file_str = ""

    if year < 9:
        file_str = "200{}-0{}".format(year, year + 1)
    elif year == 9:
        file_str = "200{}-{}".format(year, year + 1)
    else:
        file_str = "20{}-{}".format(year, year + 1)

    print("Starting loading season {}".format(file_str))
    seasons_data = data_loader.load_data(seasons=[file_str])
    print("Finished loading season {}".format(file_str))

    pbp_grouped = seasons_data.groupby("GAME_ID")
    game_data = pbp_grouped.size().to_frame(name="play_count")

    game_data["HOME TEAM ID"] = pbp_grouped.apply(lambda x: x[x["PERSON1TYPE"] == 'HOME_PLAYER']["PLAYER1_TEAM_ID"].iloc[0])
    game_data["VISITOR TEAM ID"] = pbp_grouped.apply(lambda x: x[x["PERSON1TYPE"] == 'VISITOR_PLAYER']["PLAYER1_TEAM_ID"].iloc[0])
    game_data[["HOME_PLAYER_1", "HOME_PLAYER_2", "HOME_PLAYER_3", "HOME_PLAYER_4", "HOME_PLAYER_5", "VISITOR_PLAYER_1", "VISITOR_PLAYER_2", "VISITOR_PLAYER_3", "VISITOR_PLAYER_4", "VISITOR_PLAYER_5"]] = np.nan


    for i, row in game_data.iterrows():
        errors = []

        home_team_id = row["HOME TEAM ID"]
        visitor_team_id = row["VISITOR TEAM ID"]
        
        home_players = seasons_data[(seasons_data["EVENTMSGTYPE"] == "SUBSTITUTION") & (seasons_data["GAME_ID"] == i) & (seasons_data["PERIOD"] == 1) &(seasons_data["PLAYER1_TEAM_ID"] == home_team_id)]["PLAYER1_ID"].dropna().unique()
        visitor_players = seasons_data[(seasons_data["EVENTMSGTYPE"] == "SUBSTITUTION") & (seasons_data["GAME_ID"] == i) & (seasons_data["PERIOD"] == 1) &(seasons_data["PLAYER1_TEAM_ID"] == visitor_team_id)]["PLAYER1_ID"].dropna().unique()

        # print(i)
        
        if len(home_players) < 5:
            players_1 = seasons_data[(seasons_data["GAME_ID"] == i) & (seasons_data["PERIOD"] == 1) & (seasons_data["PLAYER1_TEAM_ID"] == home_team_id)]["PLAYER1_ID"].dropna().unique()
            players_2 = seasons_data[(seasons_data["GAME_ID"] == i) & (seasons_data["PERIOD"] == 1) & (seasons_data["PLAYER2_TEAM_ID"] == home_team_id)]["PLAYER2_ID"].dropna().unique()
            players_3 = seasons_data[(seasons_data["GAME_ID"] == i) & (seasons_data["PERIOD"] == 1) & (seasons_data["PLAYER3_TEAM_ID"] == home_team_id)]["PLAYER3_ID"].dropna().unique()
            out_players = seasons_data[(seasons_data["EVENTMSGTYPE"] == "SUBSTITUTION") & (seasons_data["GAME_ID"] == i) & (seasons_data["PERIOD"] == 1) &(seasons_data["PLAYER1_TEAM_ID"] == home_team_id)]["PLAYER1_ID"].dropna().unique()
            in_players = seasons_data[(seasons_data["EVENTMSGTYPE"] == "SUBSTITUTION") & (seasons_data["GAME_ID"] == i) & (seasons_data["PERIOD"] == 1) &(seasons_data["PLAYER2_TEAM_ID"] == home_team_id)]["PLAYER2_ID"].dropna().unique()

            lineup = reduce(np.union1d, (players_1, players_2, players_3, home_players))
            lineup = np.setdiff1d(lineup, in_players)
            lineup = np.setdiff1d(lineup, out_players)
            home_players = np.union1d(lineup, out_players)
        
        if len(visitor_players) < 5:
            players_1 = seasons_data[(seasons_data["GAME_ID"] == i) & (seasons_data["PERIOD"] == 1) & (seasons_data["PLAYER1_TEAM_ID"] == visitor_team_id)]["PLAYER1_ID"].dropna().unique()
            players_2 = seasons_data[(seasons_data["GAME_ID"] == i) & (seasons_data["PERIOD"] == 1) & (seasons_data["PLAYER2_TEAM_ID"] == visitor_team_id)]["PLAYER2_ID"].dropna().unique()
            players_3 = seasons_data[(seasons_data["GAME_ID"] == i) & (seasons_data["PERIOD"] == 1) & (seasons_data["PLAYER3_TEAM_ID"] == visitor_team_id)]["PLAYER3_ID"].dropna().unique()
            out_players = seasons_data[(seasons_data["EVENTMSGTYPE"] == "SUBSTITUTION") & (seasons_data["GAME_ID"] == i) & (seasons_data["PERIOD"] == 1) &(seasons_data["PLAYER1_TEAM_ID"] == visitor_team_id)]["PLAYER1_ID"].dropna().unique()
            in_players = seasons_data[(seasons_data["EVENTMSGTYPE"] == "SUBSTITUTION") & (seasons_data["GAME_ID"] == i) & (seasons_data["PERIOD"] == 1) &(seasons_data["PLAYER2_TEAM_ID"] == visitor_team_id)]["PLAYER2_ID"].dropna().unique()

            lineup = reduce(np.union1d, (players_1, players_2, players_3, visitor_players))
            lineup = np.setdiff1d(lineup, in_players)
            lineup = np.setdiff1d(lineup, out_players)
            visitor_players = np.union1d(lineup, out_players)
        try:
            game_data.at[i, "HOME_PLAYER_1":"HOME_PLAYER_5"] = np.asarray(home_players[:5])
            game_data.at[i, "VISITOR_PLAYER_1" :] = np.asarray(visitor_players[:5])
        except:
            game_data.at[i, "HOME_PLAYER_1":"HOME_PLAYER_5"] = np.nan
            game_data.at[i, "VISITOR_PLAYER_1" :] = np.nan
            errors.append(i)

    file_name = "lineup-{}-id.csv".format(file_str)
    game_data.to_csv(file_name, index=True)

    if len(errors) > 0:
        print("CHECK ERRORS season {}".format(file_str))
        print(errors)
    else:
        print("DONE season {}".format(file_str))

Starting loading season 2000-01


NameError: name 'data_loader' is not defined

In [41]:
frames = []

for year in range(0, 19):

    file_str = ""

    if year < 9:
        file_str = "lineup-200{}-0{}-id.csv".format(year, year + 1)
    elif year == 9:
        file_str = "lineup-200{}-{}-id.csv".format(year, year + 1)
    else:
        file_str = "lineup-20{}-{}-id.csv".format(year, year + 1)

    df = pd.read_csv(file_str)
    df.set_index("GAME_ID", inplace=True)
    frames.append(df)

lineup_all = pd.concat(frames)

In [45]:
lineup_all.to_csv("lineups-all-seasons.csv", index=True)